In [2]:
import DataParser
#from DataParser import *
import pandas as pd
import importlib
importlib.reload(DataParser)
from DataParser import DataParser as dp
import pytest as pt
import ipytest
ipytest.autoconfig()
import numpy as np 

In [3]:
%%ipytest -qq
new_test = "000000000000002C08010000017B49D33298001EA917C50F16A15500000000000000000E0501FFDF9F0365AFFF2FFF06B5FFFFB6FFFF42FFFF18FFFF43FFFF0FFFFF03F1FFFFFFFFC7FFFFFFFF10FFFFFFFF000100003148"

def test_gps():
    """ Testing gps values from test string """
    imei = '1234'
    avl_obj = dp(imei)
    count = avl_obj.get_avl_count(new_test)
    result_dict = avl_obj.get_avl_data(count, new_test)
    result_dict = result_dict[0]
    x = result_dict["gps"][0]
    y = result_dict["gps"][1]
    assert [x,y] == [514398149, 253141333]

    
def test_n1():
    """ Testing n1 values from test string """
    imei = '1234'

    avl_obj = dp(imei)
    count = avl_obj.get_avl_count(new_test)
    result_dict = avl_obj.get_avl_data(count, new_test)
    result_dict = result_dict[0]
    x = result_dict["n1"]['parsed_data']
    #y = result_dict["gps"][1]
    assert x == {'ID 1': 255,
                'ID 223': 159,
                'ID 3': 101,
                'ID 175': 255,
                'ID 47': 255}

def test_n2():
    """ Testing n2 values from test string """
    imei = '1234'

    avl_obj = dp(imei)
    count = avl_obj.get_avl_count(new_test)
    result_dict = avl_obj.get_avl_data(count, new_test)
    result_dict = result_dict[0]
    x = result_dict["n2"]['parsed_data']
    #y = result_dict["gps"][1]
    assert x ==     {'ID 181': 65535,
                    'ID 182': 65535,
                    'ID 66': 65535,
                    'ID 24': 65535,
                    'ID 67': 65535,
                    'ID 15': 65535}
    
    

    

...                                                                                          [100%]


In [4]:
ipytest.run()


...                                                                                          [100%]
3 passed in 0.01s


In [5]:
df = pd.read_csv('all_data.csv', skiprows=3)
df = df.iloc[:,4:]
df_short = df[:]
list_avls = []
for val, row in df_short[['_value','imei']].iterrows():

    avl_obj = dp(row.imei)
    val = row._value.split('\"')[0]
    #print(val)
    count = avl_obj.get_avl_count(val)
    #print(count)
    list_avls.append(avl_obj.get_avl_data(count, val))

In [6]:
all_keys = []
for val in list_avls:
    for val2 in val.items():
        #print(val2)
        if val2[1]['n1']['parsed_data']:
            all_keys.extend(val2[1]['n1']['parsed_data'].keys())
        if val2[1]['n2']['parsed_data']:
            all_keys.extend(val2[1]['n2']['parsed_data'].keys())
        if val2[1]['n4']['parsed_data']:
            all_keys.extend(val2[1]['n4']['parsed_data'].keys())
        if val2[1]['n8']['parsed_data']:
            all_keys.extend(val2[1]['n8']['parsed_data'].keys())
        



In [36]:
import requests
from bs4 import BeautifulSoup as bs
r = requests.get('https://wiki.teltonika-gps.com/view/FMC130_AVL_ID')
html = r.text
parser = bs(html, 'html.parser')
rows = parser.find_all("tr")
map_avls = {}
for val in rows:
    try:
        map_avls[int(val.text.split('\n')[1])] =  val.text.split('\n')[3]
    except:
        print("Not an integer")
        
for val in set(all_keys):
    int_val = int(val.split()[1])
    print(f"This is the int {int_val} and this is equal to {map_avls[int_val]}")

Not an integer
Not an integer
Not an integer
Not an integer
Not an integer
Not an integer
Not an integer
Not an integer
This is the int 47 and this is equal to EGR Error
This is the int 181 and this is equal to GNSS PDOP
This is the int 52 and this is equal to Absolute Load Value
This is the int 46 and this is equal to Commanded EGR
This is the int 66 and this is equal to External Voltage
This is the int 175 and this is equal to Auto Geofence
This is the int 240 and this is equal to Movement
This is the int 45 and this is equal to Direct Fuel Rail Pressure
This is the int 16 and this is equal to Total Odometer
This is the int 13 and this is equal to Fuel Rate GPS
This is the int 241 and this is equal to Active GSM Operator
This is the int 246 and this is equal to Towing
This is the int 182 and this is equal to GNSS HDOP
This is the int 31 and this is equal to Engine Load
This is the int 30 and this is equal to Number of DTC
This is the int 69 and this is equal to GNSS Status
This is th

### Connecting to Database

In [20]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [29]:
# inserting data

In [47]:
##conn = psycopg2.connect()
#conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT) 
conn = psycopg2.connect()
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT) 

for dicts in list_avls[:]:
    for val in dicts.items():
        imei = val[1]['imei']
        time = val[1]['time']
        gps = val[1]["gps"]
        event = val[1]["event_id"]
        if val[1]['n1']['parsed_data']:
            n1 = val[1]['n1']["parsed_data"]
            n1_ids = f"array{[int(val.split(' ')[1]) for val in n1.keys()]}"
            n1_vals = f"array{[val for val in n1.values()]}"
        else:
            n1_ids = 'NULL'
            n1_vals = 'NULL'
            
        if val[1]['n2']['parsed_data']:
            n2 = val[1]["n2"]["parsed_data"]
            n2_ids = f"array{[int(val.split(' ')[1]) for val in n2.keys()]}"
            n2_vals = f"array{[val for val in n2.values()]}"
        else:
            n2_ids = 'NULL'
            n2_vals = 'NULL'
        
        if val[1]['n4']['parsed_data']:
            n4 = val[1]['n4']["parsed_data"]
            n4_ids = f"array{[int(val.split(' ')[1]) for val in n4.keys()]}"
            n4_vals = f"array{[val for val in n4.values()]}"
        else:
            n4_ids = 'NULL'
            n4_vals = 'NULL'
            
        if val[1]['n8']['parsed_data']:
            n8 = val[1]['n8']["parsed_data"]
            n8_ids = f"array{[int(val.split(' ')[1]) for val in n8.keys()]}"
            n8_vals = f"array{[val for val in n8.values()]}"
        else:
            n8_ids = 'NULL'
            n8_vals = 'NULL'
        #print({time})
        #print(type(time))
        #print(f"{imei},{(gps[1]/10000000, gps[0]/10000000)},{time},{event},{n1_ids}, {n1_vals}, {n2_ids}, {n2_vals}, {n4_ids}, {n4_vals}, {n8_ids}, {n8_vals}")
        table_sql = f"""insert into car_data(imei, gps, time, event_id, n1_ids, n1_values, n2_ids, n2_values, n4_ids, n4_values, n8_ids, n8_values)  
                        values({imei}, ST_GeomFromText(\'Point({gps[0]/10000000} {gps[1]/10000000})\', 4326), \'{time}\',{event},
                                                     {n1_ids}, {n1_vals}, {n2_ids}, {n2_vals}, {n4_ids}, {n4_vals}, 
                                                     {n8_ids}, {n8_vals})""" 
        
        #print(table_sql)
        cur = conn.cursor()
        cur.execute(table_sql)
conn.close()